In [1]:
import igraph as ig

import numpy as np
import os
#Visualizations
import matplotlib.pyplot as plt
import random
from datetime import datetime
import itertools

import swnHeatKernels as swnN 
from scipy import linalg
import helperFunc as hf
import swnMetrics as swn
import centrality as ce
import distributions as dstr

from scipy.stats.stats import pearsonr 
from numpy.polynomial.polynomial import polyfit
from sklearn.utils import resample


## Functions

In [2]:
def getModularityScoreMultilevel(A):
    
    #construct it so that igraph can read it, make it undirected
    g = ig.Graph(directed=False)
    #make it weighted
    g.es["weight"] = 1.0                
                
    g.add_vertices(len(A))
    ix, jx = A.nonzero()
    for i,j in zip(ix, jx):
        if i<j:
            g.add_edge(i, j, weight=A[i,j])
            
    #calculate the clusters and their modularity score    
    clusters = g.community_multilevel(weights=g.es['weight'])
    modularity_score = g.modularity(clusters.membership,weights=g.es['weight'])
    
    return modularity_score

In [3]:
#calculates Qrandom and Q after rewirings
def getQRandomQ1Q2(adjDict):
    
    repetitions = len(adjDict)
    keyTuple = list(adjDict[1].keys())
    
    Q = {}
    for keyT in keyTuple:
        Q0 = np.zeros(repetitions)
        Q1 = np.zeros(repetitions)
        Q2 = np.zeros(repetitions)
        for ind, k in enumerate(adjDict):

            #load the specific rewired matrices -random and rewired
            A0 = adjDict[k][keyT][0]
            A1 = adjDict[k][keyT][1]
            A2 = adjDict[k][keyT][2]
        
            #store them in the Q vectors
            Q0[ind] = getModularityScoreMultilevel(A0)
            Q1[ind] = getModularityScoreMultilevel(A1)
            Q2[ind] = getModularityScoreMultilevel(A2)
        
        Q[keyT] = (Q0,Q1,Q2)
               
    return Q

## Calculate Qrandom , Q4000 and Q8000 from A and save it 
### This is for a few taus but for many repetitions 1000
### we do it to get the scatter plots for typical examples of transition, modular and centralized regions

In [4]:
weightDist = ['normal','lognormal']
pRand =[0, 0.2]

directoryLoad ='data/ArandA/1000iterations/QrandQ4000Q8000/'

Q = {}
for wD in weightDist:
    for p in pRand:
        filePathLoad = directoryLoad + 'A0A1A2' +wD+'_pRand'+str(p)+'.pckl'
        ArandA1A2 = hf.loadVar(filePathLoad)
        Q[wD,p] = getQRandomQ1Q2(ArandA1A2)



In [5]:
directorySave = 'data/ModularityValues/1000iterations/QrandQ4000Q8000/'
filePathQSave = directorySave + 'Q0Q1Q2.pckl'
hf.saveVarSimple(Q, filePathQSave)  


## Calculate Qrandom , Q4000 and Q8000 from A and save it - This is for many different taus fewer reperitions (100)

In [6]:

rewiringsTuple= (4000,4000)
weightDist = ['normal','lognormal']

pRand = [0, 0.2]
tauFirst = {}
tauFirst['normal'] = [4.2, 4.15]
tauFirst['lognormal'] = [5.8, 5.5]
tauDiffSecond = np.array([-2.2,-2,-1.8,-1.6,-1.4,-1.2,-1,-0.8,-0.6,-0.4,-0.2,0,0.2,
                          0.4,0.6,0.8,1,1.2,1.4,1.6,1.8,2,2.2])

tauSecond = {}
for indP,p in enumerate(pRand):
    for wD in weightDist:
        temp = tauFirst[wD][indP] + tauDiffSecond
        tauSecond[(wD,p)] = np.around(temp,decimals=2)

    
tauTupleList = {}
for indP,p in enumerate(pRand):
    for wD in weightDist:
        tauTupleList[(wD,p)] = []   
        for ind in np.arange(len(tauSecond[(wD,p)])):
            tauTupleList[(wD,p)].append((tauFirst[wD][indP],tauSecond[(wD,p)][ind]))


repetitions = 100


In [7]:
directoryLoad ='data/ArandA/2tauRewirings/'

QAll = {}
for wD in weightDist:
    for p in pRand:
        filePathLoad = directoryLoad + 'ArandA1A2_'+wD+'_pRand'+str(p)+'.pckl'
        ArandA1A2 = hf.loadVar(filePathLoad)
        QAll[(wD,p)] = getQRandomQ1Q2(ArandA1A2)
        
        del ArandA1A2
        
directoryQSave ='data/ModularityValues/2tauRewirings/'
descr = 'Q0Q4000Q8000'

filePathQSave = directoryQSave + descr+'.pckl'
hf.saveVarSimple((QAll,tauTupleList), filePathQSave)     